a) Import bibliotek

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


ImportError: Traceback (most recent call last):
  File "c:\Programs\Python\3.11\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Procedura inicjowania biblioteki dołączanej dynamicznie (DLL) nie powiodła się.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

b) Definicja ?rodowiska MDP

In [ ]:
states = ['Healthy', 'Stable', 'Critical']
actions = ['NoTreatment', 'Standard', 'Aggressive']
state_idx = {s:i for i,s in enumerate(states)}
action_idx = {a:i for i,a in enumerate(actions)}
gamma = 0.95


c) Definicja funkcji przej?cia i nagrody

In [ ]:
state_value = {'Healthy': 2, 'Stable': 1, 'Critical': 0}
costs = {'NoTreatment': 0.0, 'Standard': 1.0, 'Aggressive': 3.0}
transition_probs = {
    'Healthy': {
        'NoTreatment': [('Healthy', 0.85), ('Stable', 0.15)],
        'Standard': [('Healthy', 0.9), ('Stable', 0.1)],
        'Aggressive': [('Healthy', 0.95), ('Stable', 0.05)]
    },
    'Stable': {
        'NoTreatment': [('Stable', 0.6), ('Critical', 0.4)],
        'Standard': [('Healthy', 0.4), ('Stable', 0.5), ('Critical', 0.1)],
        'Aggressive': [('Healthy', 0.6), ('Stable', 0.35), ('Critical', 0.05)]
    },
    'Critical': {
        'NoTreatment': [('Critical', 0.85), ('Stable', 0.15)],
        'Standard': [('Critical', 0.6), ('Stable', 0.35), ('Healthy', 0.05)],
        'Aggressive': [('Critical', 0.4), ('Stable', 0.5), ('Healthy', 0.1)]
    }
}

def step(state, action, cost_override=None):
    probs = transition_probs[state][action]
    r = random.random()
    cum = 0.0
    next_state = probs[-1][0]
    for s, p in probs:
        cum += p
        if r <= cum:
            next_state = s
            break
    cost = costs[action] if cost_override is None else cost_override
    reward = (state_value[next_state] - state_value[state]) * 10.0 - cost
    return next_state, reward


d) Implementacja tablicowego Q-learning

In [ ]:
Q = np.zeros((len(states), len(actions)))
alpha = 0.2
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay = 0.995

def select_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    return actions[int(np.argmax(Q[state_idx[state]]))]

def update_q(state, action, reward, next_state):
    s = state_idx[state]
    a = action_idx[action]
    ns = state_idx[next_state]
    target = reward + gamma * np.max(Q[ns])
    Q[s, a] = Q[s, a] + alpha * (target - Q[s, a])


e) Trening agenta Q-learning

In [ ]:
episodes = 500
max_steps = 30
q_rewards = []

for ep in range(episodes):
    state = random.choice(states)
    total = 0.0
    for t in range(max_steps):
        action = select_action(state, epsilon)
        next_state, reward = step(state, action)
        update_q(state, action, reward, next_state)
        state = next_state
        total += reward
    q_rewards.append(total)
    epsilon = max(epsilon_min, epsilon * epsilon_decay)


f) Analiza polityki i krzywej uczenia

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(q_rewards, label='Q-learning')
plt.xlabel('Epizod')
plt.ylabel('Suma nagr?d')
plt.legend()
plt.tight_layout()
plt.show()


g) Implementacja Deep Q-Network (DQN)

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Input(shape=(len(states),)),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(len(actions), activation='linear')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')
    return model

from collections import deque

class ReplayBuffer:
    def __init__(self, capacity=5000):
        self.buffer = deque(maxlen=capacity)
    def add(self, s, a, r, ns, done):
        self.buffer.append((s, a, r, ns, done))
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        s, a, r, ns, done = map(np.array, zip(*batch))
        return s, a, r, ns, done
    def __len__(self):
        return len(self.buffer)

model = build_model()
target_model = build_model()
target_model.set_weights(model.get_weights())

buffer = ReplayBuffer()
batch_size = 32
dqn_gamma = 0.95
dqn_epsilon = 1.0
dqn_epsilon_min = 0.05
dqn_epsilon_decay = 0.995
update_target_every = 10

def state_onehot(state):
    v = np.zeros(len(states))
    v[state_idx[state]] = 1.0
    return v

def dqn_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    q_vals = model.predict(state_onehot(state)[None, :], verbose=0)[0]
    return actions[int(np.argmax(q_vals))]


h) Trening DQN

In [ ]:
dqn_episodes = 400
max_steps = 30
dqn_rewards = []

for ep in range(dqn_episodes):
    state = random.choice(states)
    total = 0.0
    for t in range(max_steps):
        action = dqn_action(state, dqn_epsilon)
        next_state, reward = step(state, action)
        buffer.add(state_onehot(state), action_idx[action], reward, state_onehot(next_state), False)
        state = next_state
        total += reward
        if len(buffer) >= batch_size:
            s, a, r, ns, done = buffer.sample(batch_size)
            q_next = target_model.predict(ns, verbose=0)
            target = model.predict(s, verbose=0)
            for i in range(batch_size):
                target[i, a[i]] = r[i] + dqn_gamma * np.max(q_next[i])
            model.fit(s, target, verbose=0)
    dqn_rewards.append(total)
    dqn_epsilon = max(dqn_epsilon_min, dqn_epsilon * dqn_epsilon_decay)
    if (ep + 1) % update_target_every == 0:
        target_model.set_weights(model.get_weights())


i) Por?wnanie Q-learning vs DQN

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(q_rewards, label='Q-learning')
plt.plot(np.linspace(0, len(q_rewards)-1, len(dqn_rewards)), dqn_rewards, label='DQN')
plt.xlabel('Epizod')
plt.ylabel('Suma nagr?d')
plt.legend()
plt.tight_layout()
plt.show()


j) Explainable Reinforcement Learning ? analiza Q

In [ ]:
state_to_explain = 'Stable'
q_values = Q[state_idx[state_to_explain]]
best_action_idx = int(np.argmax(q_values))
best_action = actions[best_action_idx]
print(state_to_explain, q_values, best_action)


k) Wyja?nienie kontrfaktyczne decyzji

In [ ]:
alt_action_idx = int(np.argsort(q_values)[-2])
alt_action = actions[alt_action_idx]
q_best = q_values[best_action_idx]
q_alt = q_values[alt_action_idx]
print(best_action, alt_action, q_best - q_alt)


l) Analiza wra?liwo?ci na funkcj? nagrody

In [ ]:
def train_q_with_aggressive_cost(cost_aggr, episodes=300):
    Q_local = np.zeros((len(states), len(actions)))
    eps = 1.0
    for ep in range(episodes):
        state = random.choice(states)
        for t in range(25):
            if random.random() < eps:
                action = random.choice(actions)
            else:
                action = actions[int(np.argmax(Q_local[state_idx[state]]))]
            cost_override = cost_aggr if action == 'Aggressive' else costs[action]
            next_state, reward = step(state, action, cost_override=cost_override)
            s = state_idx[state]
            a = action_idx[action]
            ns = state_idx[next_state]
            target = reward + gamma * np.max(Q_local[ns])
            Q_local[s, a] = Q_local[s, a] + 0.2 * (target - Q_local[s, a])
            state = next_state
        eps = max(0.05, eps * 0.99)
    policy = {s: actions[int(np.argmax(Q_local[state_idx[s]]))] for s in states}
    return policy

policy_default = {s: actions[int(np.argmax(Q[state_idx[s]]))] for s in states}
policy_high_cost = train_q_with_aggressive_cost(6.0)
print(policy_default)
print(policy_high_cost)
